In [2]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 req

In [3]:
from datasets import load_dataset

In [4]:
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import load_metric


In [5]:
import pandas as pd

# Specify the path to your CSV file
file_path = '/content/etiketli_veri.xlsx'

# Read the CSV file into a DataFrame
df = pd.read_excel(file_path)

# Display the first few rows of the DataFrame
print(df.head())


                                            Metinler  \
0  Telefon kullanmaya başladığımdan beri Turkcell...   
1  Turkcell hat iptal \nhenüz bir saat olmadı Tur...   
2  Uzun yıllardır Turkcell Faturalı hat kullanıcı...   
3  Turkcell hattımın paketini yeniledim ve 2 aylı...   
4  Bilgim olmadan abonelik yaptırılıp faturama ek...   

                               sentiment  
0  Negative - Pricing and Package Issues  
1              Negative - Service Issues  
2  Negative - Pricing and Package Issues  
3  Negative - Pricing and Package Issues  
4                                Neutral  


In [11]:
# Filter and sample 2000 rows for each sentiment category
df_positive = df[df['sentiment'] == 'Positive'].sample(n=305, random_state=42)
df_Pricing = df[df['sentiment'] == 'Negative - Pricing and Package Issues'].sample(n=305, random_state=42)
df_Service = df[df['sentiment'] == 'Negative - Service Issues'].sample(n=305, random_state=42)
df_neutral = df[df['sentiment'] == 'Neutral'].sample(n=305, random_state=42)

# Combine the sampled DataFrames
df_sampled = pd.concat([df_positive, df_Pricing,df_Service, df_neutral])

# Shuffle the combined DataFrame (optional)
df_sampled = df_sampled.sample(frac=1, random_state=42).reset_index(drop=True)

# Save the new DataFrame to a CSV file
df_sampled.to_csv('/content/sampled_etiket.csv', index=False)

# Display the first few rows of the new DataFrame
print(df_sampled.head())

                                            Metinler  \
0  Turkcell'e numara taşıma yöntemi ile geçiş yap...   
1  Güne @TurkTelekom dan kurtulup, @Turkcell e ge...   
2  @Turkcell kurulduğundan beri abonesiyim hiç de...   
3  Turkcell müşteri hizmetleri ile bağlantı kurar...   
4  Turkcell internet paketim dolmadan hızım inanı...   

                               sentiment  
0  Negative - Pricing and Package Issues  
1                               Positive  
2                               Positive  
3                                Neutral  
4              Negative - Service Issues  


In [9]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

device(type='cuda')

In [15]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from datasets import load_metric

In [16]:
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy = accuracy_score(labels, preds)

    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [17]:

# Assuming you have your DataFrame (df)
# Example: df = pd.read_csv('your_dataset.csv')

# Convert the DataFrame to a Dataset object
dataset = Dataset.from_pandas(df_sampled)

# Define the label mappings for 4 classes
label2id = {
    "Positive": 0,
    "Neutral": 1,
    "Negative - Service Issues": 2,
    "Negative - Pricing and Package Issues": 3
}
id2label = {v: k for k, v in label2id.items()}

# Map the labels in the Dataset
dataset = dataset.map(lambda examples: {'label': label2id[examples['sentiment']]})

# Now your dataset has integer labels instead of text labels
df_converted = dataset.to_pandas()

# Split the DataFrame into train and test sets
train_df, test_df = train_test_split(df_converted, test_size=0.2, stratify=df_converted['label'])

# Convert pandas DataFrames to Hugging Face Dataset objects
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Initialize the tokenizer
model_name = "savasy/bert-base-turkish-sentiment-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)

# Load the pre-trained model and adjust for 4 classes
model = BertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=4,  # Set the number of output labels to 4
    ignore_mismatched_sizes=True  # Handle the size mismatch error
)

# Update model config with label mappings
model.config.label2id = label2id
model.config.id2label = id2label

# Tokenize the data
def preprocess_function(examples):
    return tokenizer(examples['Metinler'], padding='max_length', truncation=True)

encoded_train_dataset = train_dataset.map(preprocess_function, batched=True)
encoded_test_dataset = test_dataset.map(preprocess_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=8,
    weight_decay=0.01,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_test_dataset,
    #compute_metrics=lambda p: load_metric('accuracy').compute(predictions=p.predictions.argmax(-1), references=p.label_ids)
    compute_metrics=compute_metrics
)

# Fine-tune the model
trainer.train()

# Save the model
model.save_pretrained('./fine-tuned-model')
tokenizer.save_pretrained('./fine-tuned-model')


Map:   0%|          | 0/1220 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at savasy/bert-base-turkish-sentiment-cased and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([4]) in the model instantiated
- classifier.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/976 [00:00<?, ? examples/s]

Map:   0%|          | 0/244 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.369191,0.860656,0.861970,0.880336,0.860656
2,No log,0.315721,0.877049,0.876826,0.883036,0.877049
3,No log,0.393789,0.889344,0.890040,0.897971,0.889344
4,No log,0.439330,0.881148,0.881162,0.889549,0.881148
5,0.295100,0.421187,0.897541,0.897579,0.901114,0.897541
6,0.295100,0.475664,0.893443,0.893714,0.899482,0.893443
7,0.295100,0.513364,0.893443,0.893714,0.899482,0.893443
8,0.295100,0.517168,0.897541,0.897793,0.904022,0.897541


('./fine-tuned-model/tokenizer_config.json',
 './fine-tuned-model/special_tokens_map.json',
 './fine-tuned-model/vocab.txt',
 './fine-tuned-model/added_tokens.json')

In [13]:
import pandas as pd

# Excel dosyasının yolunu belirtin
excel_dosyasi = "/content/deneme.xlsx"

# Excel dosyasını okuyup DataFrame'e dönüştürmek
df = pd.read_excel(excel_dosyasi)

# DataFrame'i görüntülemek için
print(df.head())
df

                                            Metinler
0  Platinum müşteri hizmetleri oldukça ilgili ve ...
1  turkcell superonline firması taşındığım yere i...
2  turkcell ’ in star 10 gb paketini kullanmaktay...
3  turkcell kullanmadığım aktif olmayan eski hatt...
4  turkcell ev internetini 1 yıldır doğru dürüst ...


,Metinler
0,Platinum müşteri hizmetleri oldukça ilgili ve ...
1,turkcell superonline firması taşındığım yere i...
2,turkcell ’ in star 10 gb paketini kullanmaktay...
3,turkcell kullanmadığım aktif olmayan eski hatt...
4,turkcell ev internetini 1 yıldır doğru dürüst ...
5,turkcell internet paketim kendiliğinden bitiyo...
6,2024 şubat ayında almış olduğum askercell hatt...
7,temmuz ayında hattıma tanımlanan yurt dışı pak...
8,gnç uygulamasından çatlat yaptım 9 saatlik int...
9,turkcell hattımın 2 yıllık taahhüdü şükür bitt...


In [14]:
!pip install TurkishStemmer
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from TurkishStemmer import TurkishStemmer
from transformers import pipeline

stop_words = set(stopwords.words('turkish'))

# Türkçe kök bulucu
stemmer = TurkishStemmer()


In [ ]:

def preprocess_text(text):
    # Küçük harfe dönüştürme
    text = text.lower()

    # Noktalama işaretleri ve özel karakterlerin kaldırılması
    text = re.sub(r'[^\w\s]', '', text)

    # Sayıların kaldırılması
    text = re.sub(r'\d+', '', text)

    # Stop words (durdurma kelimeleri) kaldırma
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]

    # Kök bulma (stemming)
    tokens = [stemmer.stem(word) for word in tokens]

    # Kelimeleri tekrar birleştirme
    return ' '.join(tokens)


# Metinlere ön işleme adımlarını uygulama
df['cleaned_text'] = df['Metinler'].apply(preprocess_text)
print(df)

                                             Metinler  \
0   Platinum müşteri hizmetleri oldukça ilgili ve ...   
1   turkcell superonline firması taşındığım yere i...   
2   turkcell ’ in star 10 gb paketini kullanmaktay...   
3   turkcell kullanmadığım aktif olmayan eski hatt...   
4   turkcell ev internetini 1 yıldır doğru dürüst ...   
5   turkcell internet paketim kendiliğinden bitiyo...   
6   2024 şubat ayında almış olduğum askercell hatt...   
7   temmuz ayında hattıma tanımlanan yurt dışı pak...   
8   gnç uygulamasından çatlat yaptım 9 saatlik int...   
9   turkcell hattımın 2 yıllık taahhüdü şükür bitt...   
10  Turkcell'in müşteri hizmetleri hızlı ve etkili...   
11  Fiber 100mb SuperOnline kullanıcısıyım yaklaşı...   
12  TurkcellHizmet ya vereceğiniz hizmeti... ya bi...   
13  Turkcell'in sunduğu hızlı internet hizmeti ve ...   
14  Turkcell’in Paycell ödeme sistemi çok kullanış...   
15  Turkcell’in BiP uygulaması ile sevdiklerimle k...   
16  Turkcell'in Superbox intern

In [ ]:
import re
import pandas as pd
import json
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the fine-tuned sentiment model and tokenizer
model_path = '/content/fine-tuned-model'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Initialize the sentiment analysis pipeline with the fine-tuned model
sentiment_model = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# NER model loading
ner_model = pipeline("ner", model="savasy/bert-base-turkish-ner-cased")

def combine_subwords(entities):
    combined_entities = []
    current_entity = ""

    for entity in entities:
        if entity['entity'].startswith('B-') or entity['entity'].startswith('I-'):
            if entity['entity'] in ['B-ORG', 'I-ORG']:
                if entity['word'].startswith('##'):
                    current_entity += entity['word'].replace('##', '')
                else:
                    if current_entity:
                        combined_entities.append(current_entity)
                        current_entity = ""
                    current_entity = entity['word']

    if current_entity:
        combined_entities.append(current_entity)

    combined_entities = list(set(combined_entities))

    return combined_entities

def analyze_entity_sentiment(text, entities):
    results = []
    for entity in entities:
        entity_index = text.find(entity)
        if entity_index != -1:
            sentiment = sentiment_model(text)
            results.append({
                "entity": entity,
                "sentiment": sentiment[0]['label']
            })
    return results

def analyze_text(text):
    # Entity extraction
    entities = ner_model(text)

    # Combine subword tokens and filter for ORGANISATION entities
    combined_entities = combine_subwords(entities)

    # Analyze sentiment for each entity
    results = analyze_entity_sentiment(text, combined_entities)

    return results

def format_analysis(text, analysis):
    entity_list = list(set([item['entity'] for item in analysis if len(item['entity']) > 2]))
    results = [{'entity': item['entity'], 'sentiment': item['sentiment']} for item in analysis if len(item['entity']) > 2]
    return {
        'text': text,
        'results': results
    }

def save_to_json(df, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for _, row in df.iterrows():
            original_text = row['Metinler']  # Use 'cleaned_text' as per your DataFrame
            analysis = format_analysis(original_text, row['analysis'])
            json.dump(analysis, f, ensure_ascii=False, indent=4)
            f.write('\n')


# DataFrame analysis
df['analysis'] = df['cleaned_text'].apply(analyze_text)

# Save results to a JSON file
save_to_json(df, 'analysis_results.json')


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Some weights of the model checkpoint at savasy/bert-base-turkish-ner-cased were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
import shutil

# 'folder_name' klasörünü zip dosyasına dönüştürmek
shutil.make_archive('fine-tuned-model', 'zip', 'fine-tuned-model')

'/content/fine-tuned-model.zip'

In [ ]:
import shutil

shutil.make_archive('/content/fine-tuned-model', 'zip', '/content/fine-tuned-model')


'/content/fine-tuned-model.zip'

In [ ]:
from google.colab import files

files.download('/content/fine-tuned-model.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import re
import pandas as pd
import json
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the fine-tuned sentiment model and tokenizer
model_path = '/content/fine-tuned-model'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Initialize the sentiment analysis pipeline with the fine-tuned model
sentiment_model = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# NER model loading
ner_model = pipeline("ner", model="savasy/bert-base-turkish-ner-cased")

def combine_subwords(entities):
    combined_entities = []
    current_entity = ""

    for entity in entities:
        if entity['entity'].startswith('B-') or entity['entity'].startswith('I-'):
            if entity['entity'] in ['B-ORG', 'I-ORG']:
                if entity['word'].startswith('##'):
                    current_entity += entity['word'].replace('##', '')
                else:
                    if current_entity:
                        combined_entities.append(current_entity)
                        current_entity = ""
                    current_entity = entity['word']

    if current_entity:
        combined_entities.append(current_entity)

    combined_entities = list(set(combined_entities))

    return combined_entities

def analyze_entity_sentiment(text, entities, window_size=10):
    results = []
    words = text.split()

    for entity in entities:
        entity_indices = [i for i, word in enumerate(words) if entity in word]
        for index in entity_indices:
            # Get context window around the entity
            start = max(0, index - window_size)
            end = min(len(words), index + window_size + 1)
            context = " ".join(words[start:end])

            # Perform sentiment analysis on the context
            sentiment = sentiment_model(context)
            results.append({
                "entity": entity,
                "sentiment": sentiment[0]['label'],
                "context": context
            })
    return results

def analyze_text(text):
    # Entity extraction
    entities = ner_model(text)

    # Combine subword tokens and filter for ORGANISATION entities
    combined_entities = combine_subwords(entities)

    # Analyze sentiment for each entity with its surrounding context
    results = analyze_entity_sentiment(text, combined_entities)

    return results

def format_analysis(text, analysis):
    entity_list = list(set([item['entity'] for item in analysis if len(item['entity']) > 2]))
    results = [{'entity': item['entity'], 'sentiment': item['sentiment']} for item in analysis if len(item['entity']) > 2]
    return {
        'text': text,
        'results': results
    }

def save_to_json(df, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for _, row in df.iterrows():
            original_text = row['Metinler']  # Use 'cleaned_text' as per your DataFrame
            analysis = format_analysis(original_text, row['analysis'])
            json.dump(analysis, f, ensure_ascii=False, indent=4)
            f.write('\n')


# DataFrame analysis
df['analysis'] = df['cleaned_text'].apply(analyze_text)

# Save results to a JSON file
save_to_json(df, 'analysis_results2.json')


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Some weights of the model checkpoint at savasy/bert-base-turkish-ner-cased were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
import re
import pandas as pd
import json
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the fine-tuned sentiment model and tokenizer
model_path = '/content/fine-tuned-model'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Initialize the sentiment analysis pipeline with the fine-tuned model
sentiment_model = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# NER model loading
ner_model = pipeline("ner", model="savasy/bert-base-turkish-ner-cased")

def combine_subwords(entities):
    combined_entities = []
    current_entity = ""

    for entity in entities:
        if entity['entity'].startswith('B-') or entity['entity'].startswith('I-'):
            if entity['entity'] in ['B-ORG', 'I-ORG']:
                if entity['word'].startswith('##'):
                    current_entity += entity['word'].replace('##', '')
                else:
                    if current_entity:
                        combined_entities.append(current_entity)
                        current_entity = ""
                    current_entity = entity['word']

    if current_entity:
        combined_entities.append(current_entity)

    combined_entities = list(set(combined_entities))

    return combined_entities

def analyze_entity_sentiment(text, entities):
    results = []
    sentences = re.split(r'(?<=[.!?]) +', text)
    for entity in entities:
        for sentence in sentences:
            if entity in sentence:
                sentiment = sentiment_model(sentence)
                results.append({
                    "entity": entity,
                    "sentiment": sentiment[0]['label'],
                    "sentence": sentence
                })
                break  # Move to the next entity after finding the relevant sentence
    return results

def analyze_text(text):
    # Entity extraction
    entities = ner_model(text)

    # Combine subword tokens and filter for ORGANISATION entities
    combined_entities = combine_subwords(entities)

    # Analyze sentiment for each entity
    results = analyze_entity_sentiment(text, combined_entities)

    return results

def format_analysis(text, analysis):
    entity_list = list(set([item['entity'] for item in analysis if len(item['entity']) > 2]))
    results = [{'entity': item['entity'], 'sentiment': item['sentiment'], 'sentence': item['sentence']} for item in analysis if len(item['entity']) > 2]
    return {
        'text': text,
        'results': results
    }

def save_to_json(df, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for _, row in df.iterrows():
            original_text = row['Metinler']  # Use 'cleaned_text' as per your DataFrame
            analysis = format_analysis(original_text, row['analysis'])
            json.dump(analysis, f, ensure_ascii=False, indent=4)
            f.write('\n')



# DataFrame analysis
df['analysis'] = df['cleaned_text'].apply(analyze_text)

# Save results to a JSON file
save_to_json(df, 'analysis_results3.json')

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Some weights of the model checkpoint at savasy/bert-base-turkish-ner-cased were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
import re
import pandas as pd
import json
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the fine-tuned sentiment model and tokenizer
model_path = '/content/fine-tuned-model'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Check if GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the sentiment analysis pipeline with the fine-tuned model
sentiment_model = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=0 if device.type == 'cuda' else -1)

# NER model loading
ner_model = pipeline("ner", model="savasy/bert-base-turkish-ner-cased", device=0 if device.type == 'cuda' else -1)

def combine_subwords(entities):
    combined_entities = []
    current_entity = ""

    for entity in entities:
        if entity['entity'].startswith('B-') or entity['entity'].startswith('I-'):
            if entity['entity'] in ['B-ORG', 'I-ORG']:
                if entity['word'].startswith('##'):
                    current_entity += entity['word'].replace('##', '')
                else:
                    if current_entity:
                        combined_entities.append(current_entity)
                        current_entity = ""
                    current_entity = entity['word']

    if current_entity:
        combined_entities.append(current_entity)

    combined_entities = list(set(combined_entities))

    return combined_entities

def analyze_entity_sentiment(text, entities):
    results = []
    sentences = re.split(r'(?<=[.!?]) +', text)  # Split text into sentences
    for entity in entities:
        for sentence in sentences:
            if entity in sentence:
                sentiment = sentiment_model(sentence)
                results.append({
                    "entity": entity,
                    "sentiment": sentiment[0]['label'],
                    "sentence": sentence
                })
                break  # Move to the next entity after finding the relevant sentence
    return results

def analyze_text(text):
    # Entity extraction
    entities = ner_model(text)

    # Combine subword tokens and filter for ORGANISATION entities
    combined_entities = combine_subwords(entities)

    # Analyze sentiment for each entity
    results = analyze_entity_sentiment(text, combined_entities)

    return results

def format_analysis(text, analysis):
    entity_list = list(set([item['entity'] for item in analysis if len(item['entity']) > 2]))
    results = [{'entity': item['entity'], 'sentiment': item['sentiment'], 'sentence': item['sentence']} for item in analysis if len(item['entity']) > 2]
    return {
        'text': text,
        'results': results
    }

def save_to_json(df, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for _, row in df.iterrows():
            original_text = row['Metinler']  # Use 'cleaned_text' as per your DataFrame
            analysis = format_analysis(original_text, row['analysis'])
            json.dump(analysis, f, ensure_ascii=False, indent=4)
            f.write('\n')


# DataFrame analysis
df['analysis'] = df['cleaned_text'].apply(analyze_text)

# Save results to a JSON file
save_to_json(df, 'analysis_results4.json')


Some weights of the model checkpoint at savasy/bert-base-turkish-ner-cased were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
from flask import Flask, request, jsonify
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

app = Flask(__name__)

# Modeli ve tokenizer'ı yükleyin
model_path = '/content/fine-tuned-model'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    inputs = tokenizer(data['text'], return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=1).tolist()
    return jsonify({'predictions': predictions})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)








from flask import Flask, request, jsonify
from flasgger import Swagger
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

app = Flask(__name__)
Swagger(app)

# Modeli ve tokenizer'ı yükleyin
model_path = '/content/fine-tuned-model'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

@app.route('/predict', methods=['POST'])
def predict():
    """
    Predict the class of a given text
    ---
    parameters:
      - name: text
        in: body
        type: string
        required: true
        description: The text to classify
    responses:
      200:
        description: The prediction result
        schema:
          type: object
          properties:
            predictions:
              type: array
              items:
                type: integer
    """
    data = request.get_json()
    inputs = tokenizer(data['text'], return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=1).tolist()
    return jsonify({'predictions': predictions})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)



 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


ModuleNotFoundError: No module named 'flasgger'